## Загрузка датасета

In [1]:
import pandas as pd
import numpy as np

In [2]:
HABR_DATASET_PATH = 'habr.xlsx'
df_original = pd.read_excel(HABR_DATASET_PATH)

In [32]:
df_original

,Unnamed: 0,Ссылка на объявление,Дата размещения,Название,Зарплата от,Зарплата до,Валюта,Вакансия,Стаж,Требования,Город,Тип занятости,Можно удаленно,Описание,Работодатель,Ссылка на работодателя,Рейтинг,Офисы,Число сотрудников,Дата сбора данных
0,0,https://career.habr.com/vacancies/1000129552,01-04-2024 16:48:58,ИТ-редактор,40000.0,60000.0,₽,Копирайтер,Стажёр (Intern),Грамотность / Русский язык / Английский язык,NaN,Неполный рабочий день,1.0,~~~ О компании и команде\nМы готовим технологи...,Glyph media,https://glph.media,NaN,"Россия, Москва, пер. Токмаков, д.20/31, стр. 1",0,01-04-2024 20:35:58
1,1,https://career.habr.com/vacancies/1000138286,06-03-2024 10:31:40,Руководитель отдела разработки / CTO,300000.0,NaN,₽,Технический директор,NaN,Ведущий (Lead) / Linux / C# / .NET / WPF / Раз...,Пермь,Полный рабочий день,NaN,"~~~ О компании и команде\nМы – Macroscop, прод...",Macroscop,http://macroscop.com/,NaN,"Россия, Пермь, ш. Космонавтов, д. 111, корп. 27",4,01-04-2024 20:35:58
2,2,https://career.habr.com/vacancies/1000134771,20-03-2024 09:37:27,Программист 1С,NaN,NaN,NaN,Программист 1С,Средний (Middle),Разработка под 1С,NaN,Полный рабочий день,1.0,~~~ О компании и команде\nАктивно развивающаяс...,ПартКом,http://www.part-kom.ru/,NaN,"Россия, Нижний Новгород, ул. Гаражная, д.9",8,01-04-2024 20:35:58
3,3,https://career.habr.com/vacancies/1000138674,27-03-2024 13:07:52,UX/UI дизайнер удаленно middle - senior,NaN,4000.0,$,UI/UX дизайнер,Средний (Middle),Figma Design / Adobe Photoshop / UI/UX дизайн ...,NaN,Полный рабочий день,1.0,~~~ О компании и команде\nПривет! Приглашаем п...,IT Force,https://itforce.io/,NaN,"Россия, Москва, 1-й Красногвардейский пр.",2,01-04-2024 20:35:59
4,4,https://career.habr.com/vacancies/1000105132,31-03-2024 18:25:07,Разработчик ETL,NaN,NaN,NaN,Разработчик баз данных,Средний (Middle),SQL / PostgreSQL / ETL,"Москва, Санкт-Петербург",Полный рабочий день,1.0,~~~ О компании и команде\nBell Integrator – од...,Bell Integrator,http://www.bellintegrator.ru,4.68,"Беларусь, Минск, Беларусь, Минск, ул. Фабрициу...",288,01-04-2024 20:35:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3333,3333,https://career.habr.com/vacancies/1000128235,27-03-2024 15:27:14,Системный аналитик 1С,NaN,NaN,NaN,Системный аналитик,Старший (Senior),Разработка ТЗ / Agile / Автоматизация процессо...,"Воронеж, Краснодар, Ярославль",Полный рабочий день,NaN,~~~ О компании и команде\nПроект: Кредитный ко...,Совкомбанк Технологии,https://sovcombank.it/,4.69,NaN,205,01-04-2024 20:59:00
3334,3334,https://career.habr.com/vacancies/1000140564,26-03-2024 19:17:27,Middle Golang разработчик [Финтех продукт],NaN,NaN,NaN,Бэкенд разработчик,Средний (Middle),Redis / Golang / PostgreSQL / GraphQL / gRPC,NaN,Полный рабочий день,1.0,~~~ О компании и команде\nМТС Digital – единый...,МТС,http://mts.ru,NaN,"Россия, Москва, пр-т Андропова, д. 18, корп. 9...",607,01-04-2024 20:58:58
3335,3335,https://career.habr.com/vacancies/1000133157,24-03-2024 14:27:33,Системный программист С,NaN,NaN,NaN,Десктоп разработчик,Средний (Middle),C / Linux / Python / Bash,NaN,Полный рабочий день,1.0,~~~ О компании и команде\nМы Группа Астра – од...,Группа Астра,https://astralinux.ru/,4.72,"Россия, Москва, Варшавское ш., д. 26",51,01-04-2024 20:59:01
3336,3336,https://career.habr.com/vacancies/1000140411,27-03-2024 15:50:28,QA Engineer (Mobile),NaN,NaN,NaN,Инженер по ручному тестированию,NaN,SOAP / SQL / Тестирование мобильных приложений...,Москва,NaN,1.0,~~~ О компании и команде\nBELL INTEGRATOR – кр...,Bell Integrator,http://www.bellintegrator.ru,4.68,"Беларусь, Минск, Беларусь, Минск, ул. Фабрициу...",288,01-04-2024 20:58:59


In [33]:
df_original.isna().sum()

Unnamed: 0                   0
Ссылка на объявление         0
Дата размещения              0
Название                     0
Зарплата от               2846
Зарплата до               2903
Валюта                    2693
Вакансия                     0
Стаж                       785
Требования                   0
Город                     1036
Тип занятости              300
Можно удаленно            1274
Описание                     0
Работодатель                 0
Ссылка на работодателя       0
Рейтинг                   1336
Офисы                      365
Число сотрудников            0
Дата сбора данных            0
dtype: int64

In [34]:
valide_cols = [
    'Дата размещения',
    'Вакансия',
    'Стаж',
    'Зарплата от',
    'Зарплата до',
    'Город',
    'Дата сбора данных',
    'Число сотрудников',
    'Тип занятости',
    'Можно удаленно',
]
def get_valide_cols(dataset, cols):
    return dataset[cols]

In [35]:
exchange_rate_template = {
    "₽": 1,
    "$": 92.37,
    "€": 99.53,
    "₸": 0.206
}
def apply_exchange_rate(row, template=exchange_rate_template):
    row["Зарплата от"] = row["Зарплата от"] * exchange_rate_template[row["Валюта"]] if pd.notna(row["Зарплата от"]) else np.nan
    row["Зарплата до"] = row["Зарплата до"] * exchange_rate_template[row["Валюта"]] if pd.notna(row["Зарплата до"]) else np.nan
    return row

In [36]:
def get_difference_percentiles(df):
    filtered_df = df.dropna(subset=['Зарплата от', 'Зарплата до']).copy()
    filtered_df['Разница'] = filtered_df['Зарплата до'] - filtered_df['Зарплата от']
    condition = filtered_df['Разница'] > 0.8 * filtered_df['Зарплата до']
    filtered_df.loc[condition, ['Зарплата от', 'Разница']] = np.nan

    filtered_df = filtered_df.dropna(subset=['Зарплата от', 'Зарплата до'])

    filtered_df = filtered_df[filtered_df['Зарплата от'] >= 1000]
    filtered_df = filtered_df[filtered_df['Зарплата до'] >= 10000]

    filtered_df['Коэффициент разницы'] = np.where(filtered_df['Зарплата до'] != 0,
                                                      filtered_df['Разница'] / filtered_df['Зарплата до'],
                                                      np.nan)

    perc25 = filtered_df['Коэффициент разницы'].quantile(0.25)
    perc50 = filtered_df['Коэффициент разницы'].quantile(0.50)
    perc75 = filtered_df['Коэффициент разницы'].quantile(0.75)

    return perc25, perc50, perc75


def fill_na_salary(df, coef):
    result_df = df.copy()
    result_df['Зарплата до'] = result_df['Зарплата до'].fillna(result_df['Зарплата от'] / (1-coef))
    result_df['Зарплата от'] = result_df['Зарплата от'].fillna(result_df['Зарплата до'] * (1-coef))
    return result_df

In [37]:
import re

def separate_cities(df):
  work_df = df.copy()
  df_new = pd.DataFrame()
  for index in range(work_df.shape[0]):
    row = re.sub("[\(\[].*?[\)\]]", "", work_df['Город'].iloc[index])
    cities = [city.lstrip().rstrip() for city in row.split(',')]
    init_row = work_df.iloc[index].copy()
    for city in cities:
      new_row = init_row.copy()
      new_row['Город'] = city
      df_new = pd.concat([df_new, pd.DataFrame([new_row])], ignore_index=True)
  return df_new

def find_cities_around_threshold(df, threshold=75):
  cities = df['Город'].unique()
  ratios = {}
  for city in cities:
    ratios[city] = df['Город'].value_counts()[city]/df.shape[0]

  answer = []
  all_percentage = 0
  sorted_dict = {k: v for k, v in sorted(ratios.items(), key=lambda item: item[1], reverse=True)}
  for label, pct in sorted_dict.items():
      all_percentage += pct*100
      if all_percentage < threshold:
          answer.append(label)
  return answer

def create_new_cities(df, cities):
  df_new = df.copy()
  for index in range(df_new.shape[0]):
    if (df_new.iloc[index]['Город'] not in cities):
      df_new.at[index, 'Город'] = 'Малый город'
  return df_new

Удаляем строки, в которых одновременно отсутствует информация о зарплате в обоих столбцах. Заменяем пропуски нулями и делаем столбец категориальным. Заполняем столбец "Город"

In [38]:
def preprocessing_data(df):
   df_new = df.copy()

   missing_data = df_new[df["Зарплата от"].isnull() & df_new["Зарплата до"].isnull()]
   df_new = df_new.drop(missing_data.index)

   df_new['Можно удаленно'].fillna(0, inplace=True)
   df_new['Можно удаленно'] = df_new['Можно удаленно'].astype('category')

   df_new.loc[(df_new['Можно удаленно']==0) & (df_new['Город'].isnull()), 'Город'] = 'Не указано'
   df_new.loc[(df_new['Можно удаленно']==1) & (df_new['Город'].isnull()), 'Город'] = 'Удаленно'

   df_new = df_new.apply(apply_exchange_rate, axis=1)
   df_new.drop("Валюта", axis=1, inplace=True)

   perc25, perc50, perc75 = get_difference_percentiles(df_new)
   df_25 = fill_na_salary(df_new, perc25)

   df_new = separate_cities(df_new)
   cities = find_cities_around_threshold(df_new, 85)
   df_new = create_new_cities(df_new, cities)

   df_new = get_valide_cols(df_new, valide_cols)
   return df_new

In [39]:
df_prep = preprocessing_data(df_original)

In [40]:
df_prep

,Дата размещения,Вакансия,Стаж,Зарплата от,Зарплата до,Город,Дата сбора данных,Число сотрудников,Тип занятости,Можно удаленно
0,01-04-2024 16:48:58,Копирайтер,Стажёр (Intern),40000.0,60000.0,Удаленно,01-04-2024 20:35:58,0,Неполный рабочий день,1.0
1,06-03-2024 10:31:40,Технический директор,NaN,300000.0,NaN,Малый город,01-04-2024 20:35:58,4,Полный рабочий день,0.0
2,27-03-2024 13:07:52,UI/UX дизайнер,Средний (Middle),NaN,369480.0,Удаленно,01-04-2024 20:35:59,2,Полный рабочий день,1.0
3,27-03-2024 13:37:56,Фулстек разработчик,Средний (Middle),120000.0,180000.0,Удаленно,01-04-2024 20:36:01,0,Полный рабочий день,1.0
4,26-03-2024 19:06:04,Менеджер продукта,Средний (Middle),100000.0,300000.0,Удаленно,01-04-2024 20:36:03,4,Полный рабочий день,1.0
...,...,...,...,...,...,...,...,...,...,...
767,25-03-2024 16:58:00,ML разработчик,Средний (Middle),170000.0,NaN,Удаленно,01-04-2024 20:58:44,10,Полный рабочий день,1.0
768,27-03-2024 20:29:37,Бэкенд разработчик,Средний (Middle),NaN,100000.0,Удаленно,01-04-2024 20:58:47,0,Полный рабочий день,1.0
769,05-03-2024 13:13:05,DevOps-инженер,Средний (Middle),NaN,300000.0,Москва,01-04-2024 20:58:47,24,Полный рабочий день,1.0
770,01-04-2024 09:46:24,Бэкенд разработчик,Средний (Middle),120000.0,NaN,Малый город,01-04-2024 20:58:54,1,Полный рабочий день,0.0
